In [1]:
#Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

In [2]:
#read in data from CSV into DataFrame
bc_inventory_csv="Metro_invt_fs_uc_sfrcondo_sm_month.csv"
bc_inventory_df=pd.read_csv(bc_inventory_csv)

In [3]:
#remove unnecessary columns
bc_clean_inventory=bc_inventory_df.copy()
#drop RegionType, State Name, and contiguous unused Dates
bc_clean_inventory.drop(bc_clean_inventory.iloc[:,3:10], inplace=True, axis=1)
#drop Region ID and latest date
bc_clean_inventory.drop(['RegionID','2023-02-28'], inplace=True, axis=1)

In [4]:
#restrict data to only top 25 markets
bc_top_25_inventory=bc_clean_inventory.loc[(bc_clean_inventory["SizeRank"]<26),:]
bc_top_25_inventory.head()

,SizeRank,RegionName,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,...,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31
0,0,United States,1733388.0,1723097.0,1701371.0,1642145.0,1541994.0,1452603.0,1405515.0,1449965.0,...,774472.0,860326.0,946905.0,1033311.0,1092069.0,1101482.0,1085079.0,1032570.0,955889.0,884119.0
1,1,"New York, NY",91252.0,90050.0,89339.0,87186.0,81944.0,76678.0,74497.0,78192.0,...,51340.0,55710.0,58872.0,61312.0,61386.0,60262.0,58241.0,55450.0,50400.0,45916.0
2,2,"Los Angeles, CA",29874.0,30428.0,30606.0,30035.0,28252.0,26635.0,25849.0,27132.0,...,16023.0,17635.0,19187.0,21015.0,22264.0,22400.0,21802.0,20477.0,18604.0,16766.0
3,3,"Chicago, IL",49782.0,49630.0,48916.0,46495.0,42304.0,38250.0,36754.0,39387.0,...,24246.0,26775.0,28894.0,30846.0,31876.0,31682.0,30798.0,28771.0,25674.0,22770.0
4,4,"Dallas, TX",33567.0,33512.0,32589.0,31265.0,29283.0,27727.0,27057.0,28543.0,...,15053.0,16146.0,17845.0,20485.0,22365.0,22863.0,22445.0,21500.0,19985.0,18560.0


In [5]:
#Rename columns for later merge
bc_renamed_25_df=bc_top_25_inventory.copy()
#put raw column names into list
bc_column_names=list(bc_renamed_25_df.columns)
#create new list for processed names
bc_new_list=[]
#Describe Data type for renaming
bc_data_name="For Sale Inventory"
#For Loop to process column names
for bc_name in bc_column_names:
    #combine old name and data type
    bc_new_list.append(f'{bc_data_name} {bc_name}')
#replace columns with new names
bc_renamed_25_df.columns=bc_new_list
#reset SizeRank and RegionName
bc_renamed_25_df.rename(columns={"For Sale Inventory SizeRank":"SizeRank",
                                "For Sale Inventory RegionName":"RegionName"},
                          inplace=True)

In [6]:
#export final dataframe for use
bc_renamed_25_df.to_csv(f"Top 25 {bc_data_name}.csv")